In [1612]:
import random
import math
import agentpy as ap
import numpy as np
# Visualization
import seaborn as sns
import pandas as pd

In [1613]:
class RandomAgent(ap.Agent):
    """ Se mueve a celdas al azar """
    
    def setup(self):
        self.position = (0,0)  # Posición actual del agente (x, y)
        self.intention = None  # Intención actual (clean, move o wait)
        self.new_position = None  # Objetivo al moverse (x, y)
        self.is_dirty = False
        
    def see(self):
        """ Percibe su entorno """
        x, y = self.position
        self.is_dirty = self.model.is_dirty[x][y]
        self.neighbors = self.model.get_neighbors(x, y) 
    
    def next(self):
        """ Decide la próxima acción según lo que percibe """
        self.intention = "wait" # por defecto espera
        if self.is_dirty:
            self.intention = "clean"
        else:
            self.new_position = random.choice(self.neighbors)
            if self.model.is_valid_position(self.new_position):
                self.intention = "move"
                
    def action(self):
        """ Ejecuta la accion """
        x, y = self.position
        if self.intention == "clean":
            self.model.clean_cell(x, y)
        elif self.intention == "move":
            self.position = self.new_position
            
    def move(self):
        if not self.model.is_board_clean:
            self.model.total_moves += 1     
        self.see()
        self.next()
        self.action()

        

In [1614]:
class SergioAgent(ap.Agent):
    """ Empieza con una estrategia de llenado en columnas, seguido de busqueda por barrido """
    
    def setup(self):
        self.position = (0,0)  # Posición actual del agente (x, y)
        self.intention = None  # Intención actual (clean, move o wait)
        self.new_position = None  # Objetivo al moverse (x, y)
        self.is_dirty = False
        self.x_dir = 1
        self.y_dir = 1
        self.is_sharing_space = False
        self.changed_dir = 0
        self.delay = 1

        
    def see(self):
        """ Percibe su entorno """
        x, y = self.position
        self.is_dirty = self.model.is_dirty[x][y] 

        self.is_sharing_space = 0
        for agent in self.model.all_agents:
            if agent.position == self.position:
                self.is_sharing_space += 1
                if self.is_sharing_space > 1:
                    break
        
        # Removing itself!
        self.is_sharing_space -= 1
        
        x_max = len(self.model.is_dirty) - 1
        y_max = len(self.model.is_dirty[0]) - 1
        if x == 0 and x == x_max:
            self.x_dir = 0
        elif x == 0 and self.x_dir != 1: 
            self.x_dir = 1
            # raise RuntimeError
        elif x == x_max:
            self.x_dir = -1
        
        if y == 0 and y == y_max:
            self.y_dir = 0
        elif y == 0 and self.y_dir != 1 and self.delay <= 0:
            self.changed_dir = 1
            self.y_dir = 1
        elif y == y_max and self.y_dir != -1 and self.delay <= 0:
            self.changed_dir = 1
            self.y_dir = -1
        
        # Prevents double change in dir, just after moving to next column
        self.delay -= 1
            
            
    
    def next(self):
        """ Decide la próxima acción según lo que percibe """
        self.intention = "wait" # por defecto espera
        if self.is_dirty:
            self.delay = 1
            self.intention = "clean"
            return  
        
        x, y = self.position
        if self.changed_dir:
            self.new_position = (x + self.x_dir, y)
            self.changed_dir = 0
            self.delay = 1
        elif self.is_sharing_space:
            # algunos sigue barriendo vertical, otros se mueve horizontal
            possible_moves = [(x + self.x_dir, y), (x, y + self.y_dir)]
            index = np.random.choice([0, 1], p=[0.95, 0.05])
            if index == 0:
                self.delay = 1
            self.new_position = possible_moves[index]
        else: 
            self.new_position = (x, y + self.y_dir)
        
        if self.model.is_valid_position(self.new_position):
            self.intention = "move"
                
    def action(self):
        """ Ejecuta la accion """
        x, y = self.position
        if self.intention == "clean":
            self.model.clean_cell(x, y)
        elif self.intention == "move":
            self.position = self.new_position
            
    def move(self):
        if not self.model.is_board_clean:
            self.model.total_moves += 1     
        self.see()
        self.next()
        self.action()


In [1615]:
class RodrigoAgent(ap.Agent):
    
    def setup(self):
        pass
        
    def see(self):
        pass 
    
    def next(self):
        pass
                
    def action(self):
       pass
            
    def move(self):
        if not self.model.is_board_clean:
            self.model.total_moves += 1     
        self.see()
        self.next()
        self.action()


In [1616]:
class OscarAgent(ap.Agent):
    
    def setup(self):
        pass
        
    def see(self):
        pass 
    
    def next(self):
        pass
                
    def action(self):
       pass
            
    def move(self):
        if not self.model.is_board_clean:
            self.model.total_moves += 1     
        self.see()
        self.next()
        self.action()


In [1617]:
class PepeAgent(ap.Agent):
    
    def setup(self):
        pass
        
    def see(self):
        pass 
    
    def next(self):
        pass
                
    def action(self):
       pass
            
    def move(self):
        if not self.model.is_board_clean:
            self.model.total_moves += 1     
        self.see()
        self.next()
        self.action()


In [1618]:
class HectorAgent(ap.Agent):
    
    def setup(self):
        pass
        
    def see(self):
        pass 
    
    def next(self):
        pass
                
    def action(self):
       pass
            
    def move(self):
        if not self.model.is_board_clean:
            self.model.total_moves += 1     
        self.see()
        self.next()
        self.action()


In [1619]:
class CleaningModel(ap.Model):
    """ Modelo de limpieza de tablero """
    
    def generateDirtyCells(self):
        """
        Genera una matriz n x m donde un porcentaje de las celdas están marcadas como sucias (1) y el resto limpias (0).
        devuelve: Matriz n x m con celdas sucias y limpias
        """
        totalCells = self.p.n * self.p.m
    
        dirtyCellsCount = math.ceil(totalCells * (self.p.percentage_dirty / 100))

        # Inicializamos la matriz con ceros
        isDirty = [[0 for _ in range(self.p.m)] for _ in range(self.p.n)]

        # Generamos las posiciones de las celdas sucias de manera aleatoria
        allPositions = [(i, j) for i in range(self.p.n) for j in range(self.p.m)]
        dirtyPositions = random.sample(allPositions, dirtyCellsCount)

        # Marcamos las celdas seleccionadas como sucias
        for i, j in dirtyPositions:
            isDirty[i][j] = 1

        return isDirty
    
    def is_valid_position(self, position):
        """ Verifica si una posición es valida dentro de nuestro tablero """
        x, y = position
        return 0 <= x < self.p.n and 0 <= y < self.p.m
    
    def get_neighbors(self, x, y):
        """ regresa lista con todas las celdas vecinas de la posicion x, y"""
        neighbors = []
        
        for dx in range(-1, 2):  # Iterata -1, 0, 1
            for dy in range(-1, 2):  # Itera -1, 0, 1
                if not (dx == 0 and dy == 0):  # Saltamos la celda actual
                    neighbors.append((x + dx, y + dy)) 
    
        return neighbors
    
    def clean_cell(self, x, y): 
        self.is_dirty[x][y] = 0 
        self.cleaned_cells += 1
        if not(self.is_board_clean) and self.cleaned_cells == self.dirty_cells:
            self.finish_time = self.t
            self.is_board_clean = True

    
    def setup(self):
        
        self.random_agents = ap.AgentList(self, self.p.random_agents, RandomAgent)
        self.sergio_agents = ap.AgentList(self, self.p.sergio_agents, SergioAgent)
        self.rodrigo_agents = ap.AgentList(self, self.p.rodrigo_agents, RodrigoAgent)
        self.oscar_agents = ap.AgentList(self, self.p.oscar_agents, OscarAgent)
        self.pepe_agents = ap.AgentList(self, self.p.pepe_agents, PepeAgent)
        self.hector_agents = ap.AgentList(self, self.p.hector_agents, HectorAgent)
        

        # self.all_agents = self.sergio_agents + self.rodrigo_agents + self.oscar_agents + self.pepe_agents + self.hector_agents
        self.all_agents = self.random_agents + self.sergio_agents + self.rodrigo_agents + self.oscar_agents + self.pepe_agents + self.hector_agents
        # Genera matriz donde isDirty[i][j] == 1 si la celda en la fila i y columna j esta sucia
        self.is_dirty = self.generateDirtyCells()
        
        # Inicializamos la matriz con ceros
        agentCountMatrix = [[0 for _ in range(self.p.m)] for _ in range(self.p.n)]
        agentCountMatrix[0][0] = len(self.all_agents)
        
        # Variables para generar estadisticas
        self.total_moves = 0
        self.cleaned_cells = 0
        self.dirty_cells = math.ceil(self.p.n * self.p.m * (self.p.percentage_dirty / 100))
        self.is_board_clean = True if self.p.percentage_dirty == 0 else False # Es falso a menos que no haya celdas sucias
        self.finish_time = self.p.steps + 1 # Por default no termina en tiempo


    def step(self):
        self.all_agents.move()

    def update(self):
        """
        self.record('Gini Coefficient (all_agents)', gini(self.all_agents.wealth))
    """

    def end(self):
        time_taken = self.finish_time
        cleaned_percentage = (self.cleaned_cells / (self.dirty_cells)) * 100
        clean_percentage = ((self.p.n * self.p.m - self.dirty_cells + self.cleaned_cells) / (self.p.n * self.p.m)) * 100
        print()
        
        if not self.is_board_clean:
            print("La limpieza no acabo, se llego al tiempo maximo")
        else:
            print(f"Tiempo de limpieza: {time_taken} pasos")
            
        print(f"Porcentaje de celdas limpiadas: {cleaned_percentage:.2f}%")
        print(f"Porcentaje de celdas limpias: {clean_percentage:.2f}%")
        print(f"Total de movimientos: {self.total_moves}")
        
        """
        self.all_agents.record('wealth')
        """

In [1620]:
parameters = {
    'random_agents': 0,
    'sergio_agents': 10,
    'rodrigo_agents': 0,
    'oscar_agents': 0,
    'pepe_agents': 0,
    'hector_agents': 0,
    'steps': 6000,
    'n': 50,
    'm': 50,
    'percentage_dirty': 100,
    'seed': 22,
}

In [1621]:
model = CleaningModel(parameters)
results = model.run()

Completed: 6000 steps
Tiempo de limpieza: 4999 pasos
Porcentaje de celdas limpiadas: 100.00%
Porcentaje de celdas limpias: 100.00%
Total de movimientos: 4999

Run time: 0:00:00.059134
Simulation finished
